<a href="https://colab.research.google.com/github/GhazaleZe/Machine-learning-for-optimal-portfolios/blob/main/CompAss4_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q PyDrive 

In [16]:
from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
import numpy as np


def reading_data():
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

  #F-F Research data factors
  file_id = '1JqjCLhMszamzs3_ihpqCVFskcZYicXYy' 


  download = drive.CreateFile({'id': file_id})

  #F-F Research data factors daily

  file_id_d = '1In0NkXHqz-QRlfivTTDaAbE2QFCrzZt6' 


  download_daily = drive.CreateFile({'id': file_id_d})

  # predicator 

  file_id_p = '1yyVF3ilhlD-GDc08BL2k_VJfLiUsbpEF'
  
  ## reading data
  download_predictor = drive.CreateFile({'id': file_id_p})
  download.GetContentFile('F-F_Research_Data_Factors.CSV')
  FF_monthly_df  = pd.read_csv("F-F_Research_Data_Factors.CSV")
  FF_monthly_df = FF_monthly_df [FF_monthly_df['Year_Month']>='192701']
  FF_monthly_df= FF_monthly_df.dropna()
  download_daily.GetContentFile('F-F_Research_Data_Factors_daily.CSV')

  FF_daily_df  = pd.read_csv("F-F_Research_Data_Factors_daily.CSV")
  FF_daily_df = FF_daily_df.dropna()
  #FF_daily_df['YYMMDD'] = FF_daily_df['YYMMDD'].astype(int)
  FF_daily_df = FF_daily_df [(FF_daily_df['YYMMDD']>= '19270101') & (FF_daily_df['YYMMDD'] <= '20211231')]

  download_predictor.GetContentFile('PredictorData2021Monthly.csv')
  Predictore_df  = pd.read_csv("PredictorData2021Monthly.csv")

  Predictore_df = Predictore_df[Predictore_df['yyyymm']>=192701]
  # delete column csp
  Predictore_df = Predictore_df.drop('csp' , axis=1)

  return FF_daily_df, FF_monthly_df, Predictore_df



FF_daily_df, FF_monthly_df, Predictore_df = reading_data()


FF_daily_df.head()


,YYMMDD,Mkt-RF,SMB,HML,RF
150,19270103,-0.79,0.19,0.09,0.01
151,19270104,0.31,0.14,-0.66,0.01
152,19270105,0.14,0.46,-0.36,0.01
153,19270106,-0.17,-0.05,0.15,0.01
154,19270107,0.30,-0.23,0.41,0.01


In [17]:
def extract_monthly_volitility(FF_daily_df):
  df_volitility_daily = pd.DataFrame(columns=['YYYYMM', 'Var','SD'])
  #def volatility_from_daily(df):
  year_index = np.arange(1927, 2022, 1)
  month_index = [f'{i:02}' for i in range(1, 13)]
  FF_daily_df['year'] = FF_daily_df['YYMMDD'].str[:4]
  FF_daily_df['month'] = FF_daily_df['YYMMDD'].str[4:6]
  FF_daily_df['day'] = FF_daily_df['YYMMDD'].str[6:]
  #FF_daily_df.head(50)
  for year in year_index:
    for month in month_index:
      sub_set_daily = FF_daily_df[(FF_daily_df['year']==str(year)) & (FF_daily_df['month']==str(month))]
      subset_copy = sub_set_daily.copy()
      subset_copy['Mkt-RF'] = subset_copy['Mkt-RF'].astype(float).apply(lambda x: round(x**2, 3))
      sub_set_daily = subset_copy
      new_index = len(df_volitility_daily) + 1
      if month == '12':
        df_volitility_daily.loc[new_index] = [str(year+1)+'01', round(sub_set_daily['Mkt-RF'].sum()/len(sub_set_daily['Mkt-RF']),3), round(np.sqrt(sub_set_daily['Mkt-RF'].sum()/len(sub_set_daily['Mkt-RF'])),3)]
      elif month <'09':
        df_volitility_daily.loc[new_index] = [str(year)+ '0' +str(int(month)+1), round(sub_set_daily['Mkt-RF'].sum()/len(sub_set_daily['Mkt-RF']),3), round(np.sqrt(sub_set_daily['Mkt-RF'].sum()/len(sub_set_daily['Mkt-RF'])),3)]
      else:
        df_volitility_daily.loc[new_index] = [str(year)+str(int(month)+1), round(sub_set_daily['Mkt-RF'].sum()/len(sub_set_daily['Mkt-RF']),3),round(np.sqrt(sub_set_daily['Mkt-RF'].sum()/len(sub_set_daily['Mkt-RF'])),3)]

  df_volitility_daily.to_csv('volitility_daily.csv', index=False)
  return df_volitility_daily

extract_monthly_volitility(FF_daily_df)

,YYYYMM,Var,SD
1,192702,0.165,0.407
2,192703,0.098,0.312
3,192704,0.315,0.562
4,192705,0.231,0.480
5,192706,0.136,0.369
...,...,...,...
1136,202109,0.313,0.559
1137,202110,0.702,0.838
1138,202111,0.602,0.776
1139,202112,0.671,0.819


In [19]:
def extract_monthly_volitility_mean(FF_daily_df):
  df_volitility_daily = pd.DataFrame(columns=['YYYYMM', 'Var','SD'])
  #def volatility_from_daily(df):
  year_index = np.arange(1927, 2022, 1)
  month_index = [f'{i:02}' for i in range(1, 13)]
  FF_daily_df['year'] = FF_daily_df['YYMMDD'].str[:4]
  FF_daily_df['month'] = FF_daily_df['YYMMDD'].str[4:6]
  FF_daily_df['day'] = FF_daily_df['YYMMDD'].str[6:]
  #FF_daily_df.head(50)
  for year in year_index:
    for month in month_index:
      sub_set_daily = FF_daily_df[(FF_daily_df['year']==str(year)) & (FF_daily_df['month']==str(month))]
      subset_copy1 = sub_set_daily.copy()
      subset_copy1['Mkt-RF'] = subset_copy1['Mkt-RF'] - (subset_copy1['Mkt-RF'].mean())
      print (subset_copy1['Mkt-RF'].mean())
      sub_set_daily = subset_copy1

      subset_copy = sub_set_daily.copy()
      subset_copy['Mkt-RF'] = subset_copy['Mkt-RF'].astype(float).apply(lambda x: round(x**2, 3))
      sub_set_daily = subset_copy
      new_index = len(df_volitility_daily) + 1
      if month == '12':
        df_volitility_daily.loc[new_index] = [str(year+1)+'01', round(sub_set_daily['Mkt-RF'].sum()/len(sub_set_daily['Mkt-RF']),3), round(np.sqrt(sub_set_daily['Mkt-RF'].sum()/len(sub_set_daily['Mkt-RF'])),3)]
      elif month <'09':
        df_volitility_daily.loc[new_index] = [str(year)+ '0' +str(int(month)+1), round(sub_set_daily['Mkt-RF'].sum()/len(sub_set_daily['Mkt-RF']),3), round(np.sqrt(sub_set_daily['Mkt-RF'].sum()/len(sub_set_daily['Mkt-RF'])),3)]
      else:
        df_volitility_daily.loc[new_index] = [str(year)+str(int(month)+1), round(sub_set_daily['Mkt-RF'].sum()/len(sub_set_daily['Mkt-RF']),3),round(np.sqrt(sub_set_daily['Mkt-RF'].sum()/len(sub_set_daily['Mkt-RF'])),3)]

  df_volitility_daily.to_csv('volitility_daily_mean.csv', index=False)
  return df_volitility_daily

extract_monthly_volitility_mean(FF_daily_df)

-4.440892098500626e-18
2.5232341468753557e-17
-6.1679056923619804e-18
3.608224830031759e-18
4.773959005888173e-17
-2.1649348980190553e-17
-1.5543122344752193e-17
-3.289549702593056e-17
2.6645352591003756e-17
3.9968028886505634e-17
1.850371707708594e-17
0.0
1.3322676295501878e-17
9.654113257610057e-18
4.11193712824132e-17
-1.8924256101565167e-17
-1.5139404881252134e-17
-2.9890619893754213e-17
6.661338147750939e-18
-6.990293118010245e-17
0.0
-1.708035422500241e-17
-2.413528314402514e-17
-3.552713678800501e-17
1.708035422500241e-17
-1.1102230246251566e-17
1.850371707708594e-17
-8.540177112501205e-18
-2.9890619893754213e-17
6.661338147750939e-18
3.416070845000482e-17
-4.2700885562506023e-17
9.25185853854297e-18
2.049642507000289e-16
2.6122894697062506e-17
0.0
0.0
1.0092936587501423e-17
-4.056584128438072e-17
4.625929269271486e-17
2.6645352591003756e-17
7.105427357601002e-17
-5.329070518200751e-17
1.7763568394002505e-17
-7.105427357601002e-17
-2.5620531337503614e-17
-5.792467954566034e-17
3

,YYYYMM,Var,SD
1,192702,0.165,0.407
2,192703,0.063,0.251
3,192704,0.315,0.561
4,192705,0.230,0.480
5,192706,0.093,0.304
...,...,...,...
1136,202109,0.295,0.543
1137,202110,0.658,0.811
1138,202111,0.506,0.711
1139,202112,0.667,0.816
